In [5]:
import os

# import openai
import sys

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
GOOGLE_STUDIO_API_KEY = os.getenv("GOOGLE_STUDIO_API_KEY")
GOOGLE_OAUTH_API_KEY = os.getenv("GOOGLE_OAUTH_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [8]:
import weaviate

headers = {"X-OpenAI-Api-Key": OPENAI_API_KEY}
client = weaviate.connect_to_local(headers=headers)
# client = weaviate.connect_to_local()

/tmp/ipykernel_39991/3975915949.py:4: ResourceWarning: unclosed <socket.socket fd=80, family=2, type=1, proto=6, laddr=('127.0.0.1', 39522), raddr=('127.0.0.1', 8080)>
  client = weaviate.connect_to_local(headers=headers)


In [9]:
assert client.is_live()

In [10]:
import json

metainfo = client.get_meta()
print(json.dumps(metainfo, indent=2))  # Print the meta information in a readable format

{
  "hostname": "http://[::]:8080",
  "modules": {
    "generative-cohere": {
      "documentationHref": "https://docs.cohere.com/reference/chat",
      "name": "Generative Search - Cohere"
    },
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "multi2vec-clip": {
      "clip_model": {
        "_commit_hash": null,
        "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_clip-ViT-B-32/0_CLIPModel",
        "add_cross_attention": false,
        "architectures": [
          "CLIPModel"
        ],
        "bad_words_ids": null,
        "begin_suppress_tokens": null,
        "bos_token_id": null,
        "chunk_size_feed_forward": 0,
        "cross_attention_hidden_size": null,
        "decoder_start_token_id": null,
        "diversity_penalty": 0,
        "do_sample": false,
        "early_stopping": false,
        "encoder_no_repeat_ngr

In [4]:
# client.close()

## Create a Collection

In [12]:
import weaviate.classes.config as wc

client.collections.create(
    name="MovieMM",  # The name of the collection ('MM' for multimodal)
    properties=[
        wc.Property(name="title", data_type=wc.DataType.TEXT),
        wc.Property(name="overview", data_type=wc.DataType.TEXT),
        wc.Property(name="vote_average", data_type=wc.DataType.NUMBER),
        wc.Property(name="genre_ids", data_type=wc.DataType.INT_ARRAY),
        wc.Property(name="release_date", data_type=wc.DataType.DATE),
        wc.Property(name="tmdb_id", data_type=wc.DataType.INT),
        wc.Property(name="poster", data_type=wc.DataType.BLOB),
    ],
    # Define & configure the vectorizer module
    vectorizer_config=wc.Configure.Vectorizer.multi2vec_clip(
        image_fields=[
            wc.Multi2VecField(name="poster", weight=0.9)
        ],  # 90% of the vector is from the poster
        text_fields=[
            wc.Multi2VecField(name="title", weight=0.1)
        ],  # 10% of the vector is from the title
    ),
    # Define the generative module
    generative_config=wc.Configure.Generative.openai(),
)

## Import Movie Data

In [14]:
import weaviate
import pandas as pd
import requests
from datetime import datetime, timezone
import json
from weaviate.util import generate_uuid5
from tqdm import tqdm
import os
import zipfile
from pathlib import Path
import base64

# Instantiate your client (not shown). e.g.:
# client = weaviate.connect_to_local()

data_url = "https://raw.githubusercontent.com/weaviate-tutorials/edu-datasets/main/movies_data_1990_2024.json"
resp = requests.get(data_url)
df = pd.DataFrame(resp.json())

# Create a directory for the images
img_dir = Path("scratch/imgs")
img_dir.mkdir(parents=True, exist_ok=True)

# Download images
posters_url = "https://raw.githubusercontent.com/weaviate-tutorials/edu-datasets/main/movies_data_1990_2024_posters.zip"
posters_path = img_dir / "movies_data_1990_2024_posters.zip"
posters_path.write_bytes(requests.get(posters_url).content)

# Unzip the images
with zipfile.ZipFile(posters_path, "r") as zip_ref:
    zip_ref.extractall(img_dir)

# Get the collection
movies = client.collections.get("MovieMM")

# Enter context manager
with movies.batch.fixed_size(50) as batch:
    # Loop through the data
    for i, movie in tqdm(df.iterrows()):
        # Convert data types
        # Convert a JSON date to `datetime` and add time zone information
        release_date = datetime.strptime(movie["release_date"], "%Y-%m-%d").replace(
            tzinfo=timezone.utc
        )
        # Convert a JSON array to a list of integers
        genre_ids = json.loads(movie["genre_ids"])
        # Convert image to base64
        img_path = img_dir / f"{movie['id']}_poster.jpg"
        with open(img_path, "rb") as file:
            poster_b64 = base64.b64encode(file.read()).decode("utf-8")

        # Build the object payload
        movie_obj = {
            "title": movie["title"],
            "overview": movie["overview"],
            "vote_average": movie["vote_average"],
            "genre_ids": genre_ids,
            "release_date": release_date,
            "tmdb_id": movie["id"],
            "poster": poster_b64,
        }

        # Add object to batch queue
        batch.add_object(
            properties=movie_obj,
            uuid=generate_uuid5(movie["id"]),
            # references=reference_obj  # You can add references here
        )
        # Batcher automatically sends batches

680it [03:43,  3.05it/s]


In [15]:
# Check for failed objects
if len(movies.batch.failed_objects) > 0:
    print(f"Failed to import {len(movies.batch.failed_objects)} objects")
    for failed in movies.batch.failed_objects:
        print(f"e.g. Failed to import object with error: {failed.message}")

Failed to import 37 objects
e.g. Failed to import object with error: send POST request: Post "http://multi2vec-clip:8080/vectorize": context deadline exceeded (Client.Timeout exceeded while awaiting headers)
e.g. Failed to import object with error: send POST request: Post "http://multi2vec-clip:8080/vectorize": context deadline exceeded (Client.Timeout exceeded while awaiting headers)
e.g. Failed to import object with error: send POST request: Post "http://multi2vec-clip:8080/vectorize": context deadline exceeded (Client.Timeout exceeded while awaiting headers)
e.g. Failed to import object with error: send POST request: Post "http://multi2vec-clip:8080/vectorize": context deadline exceeded (Client.Timeout exceeded while awaiting headers)
e.g. Failed to import object with error: send POST request: Post "http://multi2vec-clip:8080/vectorize": context deadline exceeded (Client.Timeout exceeded while awaiting headers)
e.g. Failed to import object with error: send POST request: Post "http:/

## Search

### Image Search

In [16]:
import weaviate
import weaviate.classes.query as wq
import os


# Instantiate your client (not shown). e.g.:
# headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}  # Replace with your OpenAI API key
# client = weaviate.connect_to_local(headers=headers)


def url_to_base64(url):
    import requests
    import base64

    image_response = requests.get(url)
    content = image_response.content
    return base64.b64encode(content).decode("utf-8")


# Get the collection
movies = client.collections.get("MovieMM")

# Perform query
src_img_path = "https://github.com/weaviate-tutorials/edu-datasets/blob/main/img/International_Space_Station_after_undocking_of_STS-132.jpg?raw=true"  # Space station pic
query_b64 = url_to_base64(src_img_path)

response = movies.query.near_image(
    near_image=query_b64,
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
    return_properties=[
        "title",
        "release_date",
        "tmdb_id",
        "poster",
    ],  # To include the poster property in the response (`blob` properties are not returned by default)
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
    )  # Print the title and release year (note the release date is a datetime object)
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # Print the distance of the object from the query

# client.close()

Interstellar 2014 157336
Distance to query: 0.345

Arrival 2016 329865
Distance to query: 0.368

Gravity 2013 49047
Distance to query: 0.377

Armageddon 1998 95
Distance to query: 0.399

The Dark Knight Rises 2012 49026
Distance to query: 0.428



### Text Search

In [20]:
import weaviate
import weaviate.classes.query as wq
import os


# Instantiate your client (not shown). e.g.:
# headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}  # Replace with your OpenAI API key
# client = weaviate.connect_to_local(headers=headers)

# Get the collection
movies = client.collections.get("MovieMM")

# Perform query
response = movies.query.near_text(
    query="history",
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
    return_properties=[
        "title",
        "release_date",
        "tmdb_id",
        "poster",
    ],  # To include the poster property in the response (`blob` properties are not returned by default)
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
    )  # Print the title and release year (note the release date is a datetime object)
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # Print the distance of the object from the query

# client.close()

American History X 1998 73
Distance to query: 0.637

300 2007 1271
Distance to query: 0.662

Click 2006 9339
Distance to query: 0.665

I Am Legend 2007 6479
Distance to query: 0.669

Troy 2004 652
Distance to query: 0.670



### Keyword (BM25) Search

In [21]:
import weaviate
import weaviate.classes.query as wq
import os


# Instantiate your client (not shown). e.g.:
# headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}  # Replace with your OpenAI API key
# client = weaviate.connect_to_local(headers=headers)

# Get the collection
movies = client.collections.get("MovieMM")

# Perform query
response = movies.query.bm25(
    query="history", limit=5, return_metadata=wq.MetadataQuery(score=True)
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["title"], o.properties["release_date"].year
    )  # Print the title and release year (note the release date is a datetime object)
    print(
        f"BM25 score: {o.metadata.score:.3f}\n"
    )  # Print the BM25 score of the object from the query

# client.close()

American History X 1998
BM25 score: 2.669

A Beautiful Mind 2001
BM25 score: 1.870

Legends of the Fall 1994
BM25 score: 1.641

Hacksaw Ridge 2016
BM25 score: 1.534

Night at the Museum 2006
BM25 score: 1.509



### Hybrid Search

In [22]:
import weaviate
import weaviate.classes.query as wq
import os


# Instantiate your client (not shown). e.g.:
# headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}  # Replace with your OpenAI API key
# client = weaviate.connect_to_local(headers=headers)

# Get the collection
movies = client.collections.get("MovieMM")

# Perform query
response = movies.query.hybrid(
    query="history", limit=5, return_metadata=wq.MetadataQuery(score=True)
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["title"], o.properties["release_date"].year
    )  # Print the title and release year (note the release date is a datetime object)
    print(
        f"Hybrid score: {o.metadata.score:.3f}\n"
    )  # Print the hybrid search score of the object from the query

# client.close()

American History X 1998
Hybrid score: 1.000

300 2007
Hybrid score: 0.394

Click 2006
Hybrid score: 0.354

I Am Legend 2007
Hybrid score: 0.304

Troy 2004
Hybrid score: 0.289



### Filtered Text Search

In [23]:
import weaviate
import weaviate.classes.query as wq
import os

from datetime import datetime


# Instantiate your client (not shown). e.g.:
# headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}  # Replace with your OpenAI API key
# client = weaviate.connect_to_local(headers=headers)

# Get the collection
movies = client.collections.get("MovieMM")

# Perform query
response = movies.query.near_text(
    query="dystopian future",
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
    filters=wq.Filter.by_property("release_date").greater_than(datetime(2020, 1, 1)),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["title"], o.properties["release_date"].year
    )  # Print the title and release year (note the release date is a datetime object)
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # Print the distance of the object from the query

# client.close()

Free Guy 2021
Distance to query: 0.653

Onward 2020
Distance to query: 0.666

Elemental 2023
Distance to query: 0.669

Everything Everywhere All at Once 2022
Distance to query: 0.673

Tenet 2020
Distance to query: 0.676



/home/chris/anaconda3/envs/weaviateRAG/lib/python3.11/site-packages/weaviate/warnings.py:202: UserWarning: Con002: You are inserting the datetime object 2020-01-01 00:00:00 without a timezone. The timezone will be set to UTC.
            To use a different timezone, specify it in the datetime object. For example:
            datetime.datetime(2021, 1, 1, 0, 0, 0, tzinfo=datetime.timezone(-datetime.timedelta(hours=2))).isoformat() = 2021-01-01T00:00:00-02:00
            
  warnings.warn(
